# Predicting the amount of sleep
In the following we will be using a [Kaggle dataset for the bellabeaat fitness tracker](https://www.kaggle.com/blessingalabie/bellabeaat-fitness-tracker-device-data?select=hourlyCalories_merged.csv) to try to predict the hours of sleep for a person on a given day.

The dataset contains three tables, which contain the following data:
1. A table which has hourly data on burned calories for a personId. That is every datapoint consists of `(personId, hourlyTimestamp, caloriesBurnt)`
2. An activity data table, which contains datapoints of the form `(personId, hourlyTimestamp, totalIntensity, averageIntensity)`
3. Sleep data aggreagted in a table containing of the form `(personId, sleepDay, totalSleepRecords, totalMinutesAsleep, totalTimeInBed)`

## Preprocessing
In order to make predictions about the amount of sleep we first have to aggregate the three tables into one usable table. We try the following strategy:
1. Take the sleep data as-is
2. We sum up the calories per day and then include them in the table 
3. We sum up the total intensity of the exercise on a given day and include it.

Note the following subtetly: All sleep records are set at 12am for the whole day. Given that sleep patterns can vary over the day we will include points 2./3. for the day before the sleep datapoint as well as the day itself to make sure that all information is contained.

In [35]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay

Let's load the three datasets and bring them into the above mentioned form.
(Uncomment any of the lines below to look at the raw data)

In [2]:
hourly_calories = pd.read_csv("fitness_tracker/hourlyCalories_merged.csv")
hourly_intensities = pd.read_csv("fitness_tracker/hourlyIntensities_merged.csv")
sleep_data = pd.read_csv("fitness_tracker/sleepDay_merged.csv")
#hourly_calories.head()
#hourly_intensities.head()
#sleep_data.head()

We bring the data to the above mentioned form
1. Drop the time from the datetime-stamps in all three tables
2. Aggregate over day+personId for the calories- and activity tables
3. Join the tables to one big table
4. Look for empty fields and consider dropping those lines

In [3]:
def get_date(datetime_string):
    return datetime_string.split(" ")[0]

def get_daily_df(df, sum_column):
    return df.drop(columns=["ActivityHour"])\
        .groupby(["Id", "ActivityDate"])[sum_column]\
        .sum()\
        .reset_index()

def get_merged_df(sleep_data, daily_calories, daily_intensities):
    return sleep_data.drop(columns=["SleepDay"])\
            .merge(daily_calories, left_on=["Id", "ActivityDate"], right_on=["Id", "ActivityDate"])\
            .merge(daily_intensities, left_on=["Id", "ActivityDate"], right_on=["Id", "ActivityDate"])

hourly_calories["ActivityDate"] = hourly_calories.ActivityHour.apply(get_date)
hourly_intensities["ActivityDate"] = hourly_intensities.ActivityHour.apply(get_date)
sleep_data["ActivityDate"] = sleep_data.SleepDay.apply(get_date)

daily_calories = get_daily_df(hourly_calories, "Calories")
daily_intensities = get_daily_df(hourly_intensities, "TotalIntensity")

aggregated_df = get_merged_df(sleep_data, daily_calories, daily_intensities)

## Data quality
In total we end up with 412 rows of data. Since, for simplification, we used an inner-join to merge the data from the three tables there are no missing values we have to take care of.

In [32]:
personsInDataSet = len(aggregated_df.Id.unique())
print("Unique persons in data set: ", personsInDataSet)

#aggregated_df.describe()

#pd.set_option("display.max_rows", None, "display.max_columns", None)
aggregated_df

Unique persons in data set:  24


,Id,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed,ActivityDate,Calories,TotalIntensity
0,1503960366,1,327,346,4/12/2016,1988,429
1,1503960366,2,384,407,4/13/2016,1798,318
2,1503960366,1,412,442,4/15/2016,1745,364
3,1503960366,2,340,367,4/16/2016,1866,349
4,1503960366,1,700,712,4/17/2016,1730,318
...,...,...,...,...,...,...,...
408,8792009665,1,343,360,4/30/2016,2897,371
409,8792009665,1,503,527,5/1/2016,1963,79
410,8792009665,1,415,423,5/2/2016,2013,101
411,8792009665,1,516,545,5/3/2016,2298,156


## Defining the ML problem
As discussed before, we want to use this data to predict the total time a person slept on a given day using the other columns. To do so, we first need to define what input we use and turn that input into a suitable form for the ML algorithm.

We suspect a strong correlation of total sleep to an individual person, which is why we need to turn the personId's into a form that is readible for the ML-algorithm. We do so by converting it to a one-hot vector of the 24 persons in the data set.

In [5]:
one_hot_labels = pd.get_dummies(aggregated_df.Id, prefix='personId')
one_hot_dataframe = aggregated_df.join(one_hot_labels)

As a simplifiying assumption (see discussion at end), we will look at the rows as individual datapoints even though we suspect correlations over time. This means we drop the `ActivityDate` column for now. (We also drop the `Id` columns as we have replace it by the one-hot vectors)

In [6]:
ml_input = one_hot_dataframe.drop(columns=["Id", "ActivityDate"])

Finally, we want to split the data set into a portion we use for training and a smaller portion we use for testing. This should be done in a random, but reproduceable way.

In [7]:
random_state = 27
train_df, test_df = train_test_split(ml_input, test_size=0.1, random_state=random_state)
print("Size of training set:", len(train_df), " and testing set: ", len(test_df))

Size of training set: 371  and testing set:  42


## The ML-algorithm
Let us apply a basic neural net to this problem

In [8]:
no_of_input_columns = len(ml_input.columns) - 1
no_of_dense_layers = 3
dense_layer_units = 32
dropout_rate = 0.1
learning_rate = ExponentialDecay(initial_learning_rate=0.01, decay_rate=0.9999, decay_steps=1)

initializer = RandomNormal(stddev=0.1)

input_layer = Input(shape=(no_of_input_columns,), dtype='float32')
neural_net_layers = [input_layer]
for i in range(no_of_dense_layers):
    layer = Dense(units=dense_layer_units, activation='relu',\
                  kernel_initializer=initializer)(neural_net_layers[-1])
    neural_net_layers.append(layer)
    
neural_net_layers.append(Dropout(rate=dropout_rate)(neural_net_layers[-1]))
output_layer = Dense(units=1, activation='relu')(neural_net_layers[-1])
    
optimizer = Adam(learning_rate=learning_rate)
model = Model(input_layer, output_layer)
model.compile(optimizer=optimizer, loss='mse')

2022-02-12 14:00:04.182544: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Training the algorithm
We first split the training set into input and and output array

In [9]:
def get_input_and_output(df):
    nn_input = df.drop(columns=["TotalMinutesAsleep"]).to_numpy()
    nn_output = df.TotalMinutesAsleep.to_numpy()
    return(nn_input, nn_output)
    
(nn_input, nn_output) = get_input_and_output(train_df)

We configure to save the weights to a file in between epochs

In [10]:
filepath = "fitness_tracker.hdf5"
callbacks_list = [ModelCheckpoint(filepath, monitor='loss', verbose=0, save_best_only=True,\
                                  save_weights_only=False, mode='auto', save_freq=1)]

Finally, we are ready to train the model

In [11]:
batch_size = 32
epochs = 3000

model.fit(x=nn_input, y=nn_output, batch_size=batch_size, epochs=epochs,\
          callbacks=callbacks_list, verbose=0);
model.save_weights(filepath)

# Some notes on the choice of hyperparameters

Given the small size of the model, we can iterate over many points in hyperparameter space, such that we could find suitable values for the number of layers, their size, the number of epochs to run and the size of batches to be processed at the same point quickly.

Some problems we faced when training the model where:
1. Instability: Running the model repeatedly with the same parameters did not always result in the fit to converge. Outside of the randomness of the initial weights, this depends largely on how high the initial value of the learning rate is. Moreover, introducing a Dropout improved this.
2. The loss function does not converge properly over time, but rather fluctuates. We addressed this successfully by introducing an exponential decay in the learning rate.
3. Overfitting: This was the actual reason to introduce the Dropout mentioned in 1. it completely solve this problem.

## Testing the model
Let us briefly calculate the loss our model has on predictions on the test set:

(You can comment out the line to load the pre-trained weights)

In [82]:
#model.load_weights("fitness_tracker_demo.hdf5")
(test_input, test_output) = get_input_and_output(test_df)
model.evaluate(test_input, test_output)

2/2 [==============================] - 0s 2ms/step - loss: 302.2112


302.2112121582031

By running the model 10 times we got an average loss of about 317.

The loss is not a very intutive measure, however. So let us instead construct calculate the relative error of the prediction over the test data set, to get a better understanding on how our model performs.

In [29]:
rel_error = lambda real, predicted: np.round(abs(real/predicted - 1.0), 3)

prediction = np.round(model.predict(test_input).flatten(),0)
rel_error(test_output, prediction)
evaluation_df = pd.DataFrame({"Real": test_output,\
                             "Predicted": prediction,\
                             "RelativeError": rel_error(test_output, prediction)})
evaluation_df

,Real,Predicted,RelativeError
0,523,506.0,0.034
1,651,618.0,0.053
2,329,319.0,0.031
3,443,436.0,0.016
4,340,338.0,0.006
5,106,91.0,0.165
6,235,250.0,0.060
7,436,445.0,0.020
8,543,552.0,0.016
9,531,524.0,0.013


In [31]:
evaluation_df.describe()

,Real,Predicted,RelativeError
count,42.000000,42.000000,42.000000
mean,444.047619,436.880951,0.038833
std,128.935029,124.806427,0.037497
min,62.000000,72.000000,0.000000
25%,397.500000,383.750000,0.015000
50%,462.000000,455.000000,0.030000
75%,527.750000,508.000000,0.045750
max,651.000000,659.000000,0.165000


We can see that the average relative error lies around 4%. Given that this is a very small dataset (412 entries), this is a comparably precise prediction.

## Feature importance
Given the different columns clearly there is the suspicion that certain features, such as `TotalTimeInBed` are a very good predictor even without considering other parameters. Let us check this here.

We do so, by randomizing one feature over the test set and then compare the prediction for this modified set with the original one.

The output of the following code snippet is a list of the loss we get for our model when we shuffle a given feature. That is, if the value is very high, this means that the importance of the feature is very high.

As expected the model depends very strongly on the variable `TotalTimeInBed`. To construct a model without this feature would likely require more data and some changes to the model (see below for details).

In [108]:
def get_randomized_feature_df(df, feature):
    new_df = df.copy(deep=True)
    new_df[feature] = df[feature].sample(frac=1).values
    return new_df

def get_feature_importance(df, feature):
    shuffled_feature_df = get_randomized_feature_df(df, feature)
    (test_input, test_output) = get_input_and_output(shuffled_feature_df)
    return model.evaluate(test_input, test_output, return_dict=True, verbose=0)['loss']

a = get_feature_importance(test_df, "TotalTimeInBed")
test = lambda feature: get_feature_importance(test_df, feature)
test_df.drop(columns=["TotalMinutesAsleep"]).columns.to_series().apply(test)

TotalSleepRecords        304.485870
TotalTimeInBed         32156.130859
Calories                 626.766724
TotalIntensity           384.527710
personId_1503960366      448.205139
personId_1644430081      302.211212
personId_1844505072      879.057739
personId_1927972279      302.211212
personId_2026352035      350.017365
personId_2320127002      302.211212
personId_2347167796      303.120026
personId_3977333714      608.467896
personId_4020332650      307.493591
personId_4319703577      451.776825
personId_4388161847      306.835724
personId_4445114986      303.662537
personId_4558609924      302.211212
personId_4702921684      303.228424
personId_5553957443      302.406860
personId_5577150313      308.495544
personId_6117666160      302.211212
personId_6775888955      290.321564
personId_6962181067      325.307892
personId_7007744171      302.211212
personId_7086361926      315.425262
personId_8053475328      302.211212
personId_8378563200      303.647705
personId_8792009665      306

## Potential improvements
The way how we preprocessed our dataset completely neglected the time-series nature of it. Clearly, it is somewhat expected that there exist correlations over time (e.g. calories burnt on the days before a sleep sessions might still be a good predictor). One could try to incorporate this either simply by adding another column like `caloriesBurntOnDayBefore`, or one could use neural network architectures specialized for time series data.

Given the small size of the dataset however, we would likely not have enough data to test and train this.
